In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
#Loading the proper version of NEURON
!ln -sfn /home/jovyan/.local/nrn-7.6/ /home/jovyan/.local/nrn

In [ ]:
import os
import pkg_resources
from pkg_resources import parse_version
req_packages = [    
                    {"bluepyopt"                : {"min_version": "1.6.22", "install_version": "1.6.42"}}
                ]

def install_req_packages():
    # currently handles installations via PyPI and GitHub
    for pkg in req_packages:        
        for pkg_name, pkg_vinfo in pkg.items():
            #print("Checking for package: {}".format(pkg_name))        
            try:
                pkg_resources.get_distribution(pkg_name)        
                current_version = parse_version(pkg_resources.get_distribution(pkg_name).version)
                #print("\t{}: current version = {}".format(pkg_name, current_version))
                if not pkg_vinfo["min_version"] or current_version < parse_version(pkg_vinfo["min_version"]) or current_version > parse_version(pkg_vinfo["install_version"]):                                                
                        #print("\tInstalling another version of {}.".format(pkg_name))
                        raise
            except:            
                if "github.com" in pkg_vinfo["install_version"]:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall git+{}".format(pkg_vinfo["install_version"]))
                else:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall {}=={}".format(pkg_name, pkg_vinfo["install_version"]))                                
                #print("\t{}: installed version = {}".format(pkg_name, pkg_vinfo["install_version"]))

install_req_packages()                 

In [ ]:
from IPython.display import display, HTML

REQUIRED_VERSIONS = {
    "hbp_service_client": "1.1.0"
}

def get_version(package_name):
    """Returns version for the given package"""
    import hbp_service_client
    return hbp_service_client.__version__
    
def install_package(package_name, versions):
    """Updates the specified package."""
    print("Changing package {}".format(package_name))
    lib = "{}=={}".format(package_name, versions[package_name])
    !pip install -I --quiet $lib            
    
# Check if a package needs to be changed
reload_required = []
for package_name, required_version in REQUIRED_VERSIONS.items():
    if get_version(package_name) != required_version:
        print("{} {}".format(get_version(package_name), required_version))
        reload_required.append(package_name)
        
if reload_required:
    display(HTML("""
    <div class="alert alert-block alert-danger">Some internal libraries had to be changed...<br>
    Please follow the popup to reload this page.</div>"""))
    
    # Change all package that requires change
    for package in reload_required:
        install_package(package, REQUIRED_VERSIONS)
    
    display(HTML("""<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
    Jupyter.notebook.dirty = false; window.location.reload(); })</script>"""))
else:
    display(HTML("""
    <div class="alert alert-block alert-success">Setup for this notebook is correct.</div>"""))

In [ ]:
from ipywidgets import widgets
from IPython.display import *
from ipywidgets import *
from IPython.display import HTML,Javascript, display
import pprint
import requests
import os
import re
import shutil
os.chdir("/home/jovyan")
if os.path.isdir('BPOPTANALYSIS')==True:
    shutil.rmtree('BPOPTANALYSIS')
if os.path.isdir('BPOPTANALYSIS')==False:
    !mkdir BPOPTANALYSIS
os.chdir("BPOPTANALYSIS")
clients = get_hbp_service_client()
from IPython.display import HTML, Javascript, display
h0=HTML('''<style>
    .widget-label { min-width: 17ex !important;}
</style>''')
#display(h0)

javascript2 = """
<script type="text/Javascript">
document.styleSheets[0].insertRule(".widget-listbox.form-control { width: auto }", 1);
document.styleSheets[0].insertRule(".form_login > label { width: 30% }", 1);
document.styleSheets[0].insertRule(".form_login > label + input { width: 70% }", 1);
</script>
"""
h1=HTML(javascript2)
h1.display='none'
#display(h1)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%capture
URL=('https://object.cscs.ch/v1/AUTH_c0a333ecf7c045809321ce9d9ecdfdea/'
'hippocampus_optimization/'
'bsp_data_repository-4.0.5_Python3.tar.gz')
!wget "{URL}"
!tar -zxvf bsp_data_repository-4.0.5_Python3.tar.gz

In [ ]:
import os
import shutil
diff = lambda l1,l2: [x for x in l1 if x not in l2]
os.chdir('bsp_data_repository-4.0.5_Python3')
os.chdir('optimizations')
FD=diff(os.listdir('.'),['README'])
for i in range(len(FD)):
    shutil.copytree(FD[i],os.path.join('..','..',FD[i]))
os.chdir('..')
os.chdir('..')
clients = get_hbp_service_client()

In [ ]:
clients = get_hbp_service_client()
import logging
logger = logging.getLogger()
logging.disable(logging.WARNING)
from ipywidgets import widgets
from IPython.display import *
from ipywidgets import *
import os
import zipfile
import shutil
import json
import bluepyopt
import pickle

import matplotlib.pyplot as plt
import operator
import tarfile
import requests
import io
import base64
import matplotlib
def plot_to_html(plt):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    return """<img src='data:image/png;base64,{}'/>""".format(base64.b64encode(buf.getvalue()).decode('ascii'))

FS=['GitHub', 'Storage']
c1=Select(description='Files are located on', options=FS,layout=Layout(height='100px',width='50%'))
display(c1)
w3=Select(description='optimizations', options=sorted(FD),layout=Layout(height='100px',width='50%'))
display(w3)
import neurom as nm
from neurom import viewer
os.chdir(w3.value)
for files in os.listdir('.'):
    if files.endswith('.zip'):
        zip_ref = zipfile.ZipFile(files, 'r')
        zip_ref.extractall()
        zip_ref.close() 
os.chdir(w3.value)
os.chdir('morphology')
for files in os.listdir('.'):
    if files.endswith(".asc"):
        nrn = nm.load_neuron(files);
%matplotlib agg
plt,ax=viewer.draw(nrn, mode='2d')
plt.savefig("{}.png".format(w3.value))
plt.clear()
%matplotlib inline
file = open("{}.png".format(w3.value), "rb")
image = file.read()
out1=widgets.Image(value=image,format='png',width=400,height=500,)
display(out1)
os.chdir('..')
os.chdir('..')
os.chdir('..') 
out=widgets.Output()
display(out)

def on_w3_clicked(change):
    out1.layout.display='none'
    with out:
        clear_output()
    %matplotlib agg
    BTV.disabled=True
    BTS.disabled=True
    os.chdir("/home/jovyan/BPOPTANALYSIS")
    if c1.value=='GitHub':
        os.chdir(w3.value)
        for files in os.listdir('.'):
            if files.endswith('.zip'):
                zip_ref = zipfile.ZipFile(files, 'r')
                zip_ref.extractall()
                zip_ref.close() 
        os.chdir(w3.value)
        os.chdir('morphology')
        for files in os.listdir('.'):
            if files.endswith(".asc"):
                nrn = nm.load_neuron(files);
        plt,ax=viewer.draw(nrn, mode='2d');
        for files in os.listdir('.'):
            if files.endswith('.png'):
                os.remove(files)
        plt.savefig("{}.png".format(w3.value))
        plt.clear()
        %matplotlib inline
        file = open("{}.png".format(w3.value), "rb")
        image = file.read()
        out1.value=image
        out1.layout.display=''
        os.chdir('..')
        os.chdir('..')
        os.chdir('..')        
    else:
        if len(w3.options)!=0:
            clients = get_hbp_service_client()
            hdr = oauth.get_token()
            my_url = 'https://services.humanbrainproject.eu/idm/v1/api/user/me'
            headers = {'Authorization':'Bearer ' + hdr}
            res = requests.get(my_url, headers = headers)
            if os.path.isdir(w3.value)==False:
                os.mkdir(w3.value)
            os.chdir(w3.value)
            if clients.storage.exists(os.path.join(get_collab_storage_path(),'BluePyOptAll',\
                                                   'resultsNSG',str(res.json()['username']),\
                                                   w3.value))==True:
                for files in clients.storage.list(os.path.join(get_collab_storage_path(),'BluePyOptAll',\
                                                               'resultsNSG',str(res.json()['username']),\
                                                               w3.value)):
                    clients.storage.download_file(os.path.join(get_collab_storage_path(),'BluePyOptAll',\
                                                               'resultsNSG',str(res.json()['username']),\
                                                               w3.value,files), files)
            if clients.storage.exists(os.path.join(get_collab_storage_path(),'BluePyOptOne','resultsNSG',\
                                                   str(res.json()['username']),\
                                                   w3.value))==True:
                for files in clients.storage.list(os.path.join(get_collab_storage_path(),'BluePyOptOne',\
                                                               'resultsNSG',str(res.json()['username']),\
                                                               w3.value)):
                    clients.storage.download_file(os.path.join(get_collab_storage_path(),'BluePyOptOne',\
                                                               'resultsNSG',str(res.json()['username']),\
                                                               w3.value,files), files)
            if clients.storage.exists(os.path.join(get_collab_storage_path(),'results',w3.value))==True:
                for files in clients.storage.list(os.path.join(get_collab_storage_path(),'results',w3.value)):
                    clients.storage.download_file(os.path.join(get_collab_storage_path(),'results',\
                                                               w3.value,files), files)
            # ========================================== SERVICE ACCOUNT =================================================
            
            if clients.storage.exists(os.path.join(get_collab_storage_path(),'resultsPizdaint-SA',w3.value))==True:
                for files in clients.storage.list(os.path.join(get_collab_storage_path(),'resultsPizdaint-SA',w3.value)):
                    clients.storage.download_file(os.path.join(get_collab_storage_path(),'resultsPizdaint-SA',\
                                                               w3.value,files), files)        
            
            # ============================================================================================================
            if any(File.endswith(".tar.gz") for File in os.listdir(".")):
                ff = tarfile.open("output.tar.gz")
                ff.extractall('OUTPUTBPOPT')
                ff.close()
                os.chdir('OUTPUTBPOPT')
            else:
                zip_ref = zipfile.ZipFile('output.zip', 'r')
                zip_ref.extractall('output')
                zip_ref.close() 
                os.chdir('output')
            for files in os.listdir('.'):
                if os.path.isdir(files):
                    foldername1=files
            os.chdir(foldername1)  
            os.chdir('morphology')
            for files in os.listdir('.'):
                if files.endswith(".asc"):
                    nrn = nm.load_neuron(files);
            plt,ax=viewer.draw(nrn, mode='2d');
            for files in os.listdir('.'):
                if files.endswith('.png'):
                    os.remove(files)
            plt.savefig("{}.png".format(w3.value))
            plt.clear()
            %matplotlib inline
            file = open("{}.png".format(w3.value), "rb")
            image = file.read()
            out1.value=image
            out1.layout.display=''
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
    BTV.disabled=False
    
w3.observe(on_w3_clicked,names='value')    


def on_c1_clicked(change):
    out1.layout.display='none'
    with out:
        clear_output()
    BTV.disabled=True
    BTS.disabled=True
    w3.layout.display='none'
    if c1.value=='GitHub':
        w3.options=sorted(FD)
        w3.layout.display=''
        BTV.disabled=False
    else:
        clients = get_hbp_service_client()
        hdr = oauth.get_token()
        my_url = 'https://services.humanbrainproject.eu/idm/v1/api/user/me'
        headers = {'Authorization':'Bearer ' + hdr}
        res = requests.get(my_url, headers = headers)
        w3list=[]
        if clients.storage.exists(os.path.join(get_collab_storage_path(),'BluePyOptAll','resultsNSG',\
                                               str(res.json()['username'])))==True:
            w3list.extend(clients.storage.list(os.path.join(get_collab_storage_path(),'BluePyOptAll',\
                                                            'resultsNSG',str(res.json()['username']))))            
        if clients.storage.exists(os.path.join(get_collab_storage_path(),'BluePyOptOne','resultsNSG',\
                                               str(res.json()['username'])))==True:
            w3list.extend(clients.storage.list(os.path.join(get_collab_storage_path(),'BluePyOptOne',\
                                                            'resultsNSG',str(res.json()['username']))))  
        if clients.storage.exists(os.path.join(get_collab_storage_path(),'results'))==True:
            w3list.extend(clients.storage.list(os.path.join(get_collab_storage_path(),'results')))  
        # ========================================== SERVICE ACCOUNT =================================================
        
        if clients.storage.exists(os.path.join(get_collab_storage_path(),'resultsPizdaint-SA'))==True:
            w3list.extend(clients.storage.list(os.path.join(get_collab_storage_path(),'resultsPizdaint-SA')))
        
        # ============================================================================================================
        w3list2=[]
        for file in w3list:
            w3list2.append(file[file.find('/')+1:len(file)])
        w3list=w3list2
        if len(w3list)!=0:
            w3.options=sorted(w3list)
            w3.layout.display=''
            BTV.disabled=False
        else:
            w3.options=[]
            w3.layout.display='none'
            with out:
                clear_output()
            out1.layout.display='none'
        w3.value=w3.options[0]
                
c1.observe(on_c1_clicked,names='value')

import plotly
plotly.offline.init_notebook_mode()
from plotly.offline import iplot, plot
from plotly.graph_objs import graph_objs as go
from plotly.graph_objs import Scatter, Layout
import plotly.plotly as py
from plotly import tools 
     
def runanalysis2(_):
    BTV.disabled=True
    BTS.disabled=True
    if c1.value=='GitHub':
        os.chdir(w3.value)
        data=json.load(open(w3.value+'_meta.json'))
        os.chdir(w3.value)
        os.chdir('figures')
        with out:
            display(IFrame(os.path.join('.','BPOPTANALYSIS',w3.value,w3.value,'figures',\
                                        'neuron_evolution'+\
                                        '_'+data['best_run'][0:data['best_run'].find('.pkl')]+'.pdf'),\
                           width=600, height=300))
            display(IFrame(os.path.join('.','BPOPTANALYSIS',w3.value,w3.value,'figures',\
                                        'neuron_objectives'+\
                                        '_'+data['best_run'][0:data['best_run'].find('.pkl')]+'.pdf'),\
                           width=600, height=300))
            display(IFrame(os.path.join('.','BPOPTANALYSIS',w3.value,w3.value,'figures',\
                                        'neuron_responses'+\
                                        '_'+data['best_run'][0:data['best_run'].find('.pkl')]+'.pdf'),\
                           width=600, height=300))
        os.chdir('..')
        os.chdir('..')
        os.chdir('..')   
        BTS.disabled=False
    else:
        if len(w3.options)!=0:
            os.chdir(w3.value)
            for files in os.listdir('.'):
                if os.path.isdir(files):
                    foldername1=files
            os.chdir(foldername1)
            for files in os.listdir('.'):
                if os.path.isdir(files):
                    foldername2=files
            os.chdir(foldername2)
            import model
            if not(any(os.path.isdir(File) and File.endswith('figures') for File in os.listdir("."))):
                os.mkdir('figures')                
            os.chdir('figures')
            for files in os.listdir('.'):
                os.remove(files)
            os.chdir('..')
            os.chdir('checkpoints')
            if not os.listdir('.'):
                BTS.disabled=True
                js = "<script>alert('No pkl file in the checkpoints folder');</script>"
                display(HTML(js))
                os.chdir('..')
                os.chdir('..')
                os.chdir('..')
                os.chdir('..')
            else:
                if 'checkpoint.pkl' not in os.listdir('.'):
                    for files in os.listdir('.'):
                        if files.endswith('pkl'):
                            shutil.copy(files,'checkpoint.pkl')
                os.chdir('..')
                myvar=!nrnivmodl mechanisms
                if os.path.isdir('r_0')==True:
                    shutil.rmtree('r_0')
                os.mkdir('r_0')
                myvar=!python opt_neuron.py --analyse --checkpoint ./checkpoints/
                os.chdir('figures')
                with out:
                    display(IFrame(os.path.join('.','BPOPTANALYSIS',w3.value,foldername1,\
                                                foldername2,'figures','neuron_evolution_checkpoint_0-.pdf'),\
                                   width=600, height=300))
                    display(IFrame(os.path.join('.','BPOPTANALYSIS',w3.value,foldername1,\
                                                foldername2,'figures','neuron_objectives_checkpoint_0-.pdf'), \
                                   width=600, height=300))
                    display(IFrame(os.path.join('.','BPOPTANALYSIS',w3.value,foldername1,\
                                                foldername2,'figures','neuron_responses_checkpoint_0-.pdf'), \
                                   width=600, height=300))
                os.chdir('..')
                os.chdir('..')
                os.chdir('..')
                os.chdir('..')
                BTS.disabled=False
        else:
            js = "<script>alert('No optimizations to analyse');</script>"
            display(HTML(js))
        
    
def saveanalysis(_):
    BTV.disabled=True
    BTS.disabled=True
    clients = get_hbp_service_client()
    storeto_path1 = os.path.join(get_collab_storage_path(),"BluePyOptAnalysis")
    if not clients.storage.exists(storeto_path1):
        clients.storage.mkdir(storeto_path1)
    hdr = oauth.get_token()
    my_url = 'https://services.humanbrainproject.eu/idm/v1/api/user/me'
    headers = {'Authorization':'Bearer ' + hdr}
    res = requests.get(my_url, headers = headers)
    storeto_path2 = os.path.join(storeto_path1,str(res.json()['username']))
    if not clients.storage.exists(storeto_path2):
        clients.storage.mkdir(storeto_path2)        
    storeto = os.path.join(storeto_path2,w3.value)
    if not clients.storage.exists(storeto):
        clients.storage.mkdir(storeto)
        if c1.value=='GitHub':
            os.chdir(w3.value)
            data=json.load(open(w3.value+'_meta.json'))
            os.chdir(w3.value)
            os.chdir('figures')
            f=str('neuron_evolution'+'_'+data['best_run'][0:data['best_run'].find('.pkl')]+'.pdf')
            clients.storage.upload_file(f,os.path.join(storeto,f),'pdf');
            f=str('neuron_objectives'+'_'+data['best_run'][0:data['best_run'].find('.pkl')]+'.pdf')
            clients.storage.upload_file(f,os.path.join(storeto,f),'pdf');
            f=str('neuron_responses'+'_'+data['best_run'][0:data['best_run'].find('.pkl')]+'.pdf')
            clients.storage.upload_file(f,os.path.join(storeto,f),'pdf');
            os.chdir('..')
            os.chdir('checkpoints')
            f=str('cell'+'_'+data['best_run'][0:data['best_run'].find('.pkl')]+'_0'+'.hoc')
            clients.storage.upload_file(f,os.path.join(storeto,f),'hoc');
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
        else:
            os.chdir(w3.value)
            for files in os.listdir('.'):
                if os.path.isdir(files):
                    foldername1=files
            os.chdir(foldername1)
            for files in os.listdir('.'):
                if os.path.isdir(files):
                    foldername1=files
            os.chdir(foldername1)  
            os.chdir('figures')
            f='neuron_evolution_checkpoint_0-.pdf'
            clients.storage.upload_file(f,os.path.join(storeto,f),'pdf');
            f='neuron_objectives_checkpoint_0-.pdf'
            clients.storage.upload_file(f,os.path.join(storeto,f),'pdf');
            f='neuron_responses_checkpoint_0-.pdf'
            clients.storage.upload_file(f,os.path.join(storeto,f),'pdf');
            os.chdir('..')
            os.chdir('checkpoints')
            f='cell_checkpoint_0.hoc'
            clients.storage.upload_file(f,os.path.join(storeto,f),'hoc');
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
BTV = Button()
BTV.description = 'View analysis'
BTV.background_color='gainsboro'
BTV.border_color='black'
BTV.on_click(runanalysis2)
display(BTV)

BTS = Button()
BTS.description = 'Save analysis'
BTS.background_color='gainsboro'
BTS.border_color='black'
BTS.on_click(saveanalysis)
BTS.disabled=True
display(BTS)

In [ ]:
from IPython.display import HTML
HTML('''
<form action="javascript:code_toggle()">
<input id="toggle-code-button" type="submit" value="Click here to toggle on/off the raw code."></form>
<script>
code_show=true;
console.log(code_show)
$('.widget-area .prompt .close').hide()
function code_toggle() {
 console.log(code_show)
 var list = document.getElementsByClassName('cell code_cell rendered unselected');
 var list_index = []
 for (var i = 0; i < list.length; i++) {
    var curr_des = [];
    curr_wrapper = $(list[i]).find(".output_wrapper");
    curr_area = $(list[i]).find(".output_area");
    if (curr_wrapper.length > 0 && curr_area.length == 0){
    list_index.push(i)
    } else if (curr_wrapper.length == 0 && curr_area.length == 0){
    list_index.push(i)
    };
 }
 console.log(list.length)
 console.log(list_index)
 for (var j = 0; j< list_index.length; j++){
 console.log(list[list_index[j]].style.display)
 }
 if (code_show){
 $('div.input').hide();
 for (var j = 0; j< list_index.length; j++){
    list[list_index[j]].style.display = "none"
    }
 } else {
 $('div.input').show();
 for (var j = 0; j< list_index.length; j++){
    list[list_index[j]].style.display = "block"
    }
 }
 code_show = !code_show
 console.log(code_show)

 }
document.getElementById("toggle-code-button").click(); 
</script>
''')